# Projekt 2 - Steam Sale!

Med:

- BeautifulSoup
- Pandas


## DISCLAIMER!

Inte snyggaste och mest effektiva koduppbyggnad -> resultat
# MEN
jag lärde mig mycket på vägen och hur mycket lättare man kunde göra det för sig själv genom att söka efter andra element än man trodde man skulle söka (parents istället för children i vissa fall osv). BeautifulSoup känns mycket mera bekvämt och tryggt att arbeta med nu nära nästan avslutad produkt :D

Men då stötte jag på vissa tillfällen då min ursprungliga soup modifierades även fast jag assignade till nya variabler/listor? ;o (t.ex. när jag extract():ar bort vissa tags)

Så jag har varit lite rädd att modifiera och snygga till uppbyggnaden (och förstås för att jag är lat,eller snarare tidsbrist och har annat att göra hehe ;)

Så använder jag inte heller Markup-cellerna så som du specifierat i beskrivningarna, utan jag använder dom mestadels för att ge rubriker över de olika delmomenten för att göra det lättare för dig att urskilja vad som händer var, <b>DO NOT REJECT ME PLS :'(</b>

## 1. Data Scraping och Data Munging

Vi laddar ner data från Steams sidor av erbjudanden på spel

In [13]:
# moduler
import requests
import pandas as pd
import datetime
import re
import os.path
from bs4 import BeautifulSoup as bs

### Vi hämtar datan från Steams sidor på erbjudanden

In [14]:
#req = requests.get("https://store.steampowered.com/search/?specials=1&page=1")
page = ""
for i in range(1,6):
    url = "https://store.steampowered.com/search/?specials=1&page=%s"%i
    req = requests.get(url)
    page += req.text
    soup = bs(page, "html.parser")
    print(i)
    print(url)


# satt till i efterhand, bry er inte något extra om denna variabel
allTitles = soup.findAll("div", {"class", "responsive_search_name_combined"})

1
https://store.steampowered.com/search/?specials=1&page=1
2
https://store.steampowered.com/search/?specials=1&page=2
3
https://store.steampowered.com/search/?specials=1&page=3
4
https://store.steampowered.com/search/?specials=1&page=4
5
https://store.steampowered.com/search/?specials=1&page=5


### Vi skapar vår tabell dit vi lägger önskad information med respektive kolumner

In [15]:
# skapar också vår variabel som vi sätter inuti kolumnen "Tid"
tid = datetime.date.today()
tid = tid.strftime("%d.%m.%Y")

offersDf = pd.DataFrame(columns=["Spelnamn", 
                                 "Rating", 
                                 "#Reviews", 
                                 "Rabatt%", 
                                 "Pris", 
                                 "OrdinariePris", 
                                 "Utgivningsår", 
                                 "Win", "Lin", "OSX", "Tid"])
offersDf

,Spelnamn,Rating,#Reviews,Rabatt%,Pris,OrdinariePris,Utgivningsår,Win,Lin,OSX,Tid


### Egna funktioner för data munging

#### Scrapear ur datan från vår soppa, funktioner i ordningsföljd:

- Logik för betygsättning till sifferbaserad betygsättning (skulle vilja göra om det till +4, +3, +2, +1, 0, -1, -2, -3, -4, men nåjaa de fungerar så här också

- Hämtar totala mängden recensioner
- Samma för produktpriser
- Samma för rabatt
- Samma för ordinariepris
- Samma för utgivningsdatum

In [16]:
# funktion för att returnera sifferbaserad betygsättning
# Ganska så dålig lösning, men fortfarande en lösning! ;) 
def reviewToNum(string):
    """
    Returns number-based review on a Steam Product.
    
    Takes the string-based review of a product as a parameter and converts it to a number.
    ----------------------------
    Overwhelmingly Positive = 10 
    Very Positive = 9
    Positive = 8
    Mostly Positive = 7
    Mixed = 6
    Mostly Negative = 5
    Negative = 4
    Very Negative = 3
    Overwhelmingly Negative = 2
    """
    reviewList = []
    reviewList = soup.findAll("div", {"class", "responsive_search_name_combined"})
    
    # don't bash me for this "solution" mr. mango :<
    for i in range(len(reviewList)):
        
        # Also checks for non-available values
        if( reviewList[i].find("span", {"class", "search_review_summary"}) == None ):
            reviewList[i] = "Unavailable"
        
        # Then convert the list elements to strings
        reviewList[i] = str(reviewList[i])
        
        if( reviewList[i].find("Overwhelmingly Positive") != -1 ):
            reviewList[i] = "10"
            
        if( reviewList[i].find("Very Positive") != -1 ):
            reviewList[i] = "9"
            
        if( reviewList[i].find("Mostly Positive") != -1 ):
            reviewList[i] = "7"
        
        if( reviewList[i].find("Positive") != -1 ):
            reviewList[i] = "8"
        
        if( reviewList[i].find("Mixed") != -1 ):
            reviewList[i] = "6"
        
        if( reviewList[i].find("Mostly Negative") != -1 ):
            reviewList[i] = "5"
            
        if( reviewList[i].find("Very Negative") != -1 ):
            reviewList[i] = "3"
            
        if( reviewList[i].find("Overwhelmingly Negative") != -1 ):
            reviewList[i] = "2"
            
        if( reviewList[i].find("Negative") != -1 ):
            reviewList[i] = "4"
            
    return reviewList

In [17]:
def getTotalReviews():
    
    """
    Gets total reviews per product
    
    Filters out the total reviews of a product and returns the (list of) results
    """
    
    totalReviews = soup.findAll("div", {"class", "col search_reviewscore responsive_secondrow"})
    
    for i in range(len(totalReviews)):
        if(totalReviews[i].find("span", {"class", "search_review_summary"}) == None ):
            totalReviews[i] = "Unavailable"
            
        totalReviews[i] = str(totalReviews[i])
        totalReviews[i] = re.sub("[^0-9]","", totalReviews[i])
        
        totalReviews[i] = totalReviews[i][2:]
        
    return totalReviews

In [18]:
def getProductPrice():
    
    """
    Get the product prices.
    
    Gets the product prices and returns a list of the prices
    """
    
    priceList = []
    priceList = soup.findAll("div", {"class", "col search_price_discount_combined responsive_secondrow"})

    for i in range(len(priceList)):
        if(priceList[i].find("strike")):
            priceList[i].strike.extract()
        
        if(priceList[i].find("span")):
            priceList[i].span.extract()

        if(priceList[i].findAll("div", {"class", "col search_price discounted responsive_secondrow"})):
            priceList[i] = priceList[i].text
            priceList[i] = re.sub("[^0-9,€]", "", priceList[i])
        else:
            priceList[i] = "Unavailable"
            
    return priceList

In [19]:
def getOrigPrice():
    """
    Get the original product price before respective discount.
    
    Gets the original price of the product and returns a list of prices
    """
    
    origPriceList = []
    origPriceList = soup.findAll("div", {"class", "col search_price_discount_combined responsive_secondrow"})
        
    for i in range(len(origPriceList)):
        if(origPriceList[i].find("strike") == None):
            origPriceList[i] = "Unavailable"
        else:
            origPriceList[i] = origPriceList[i].find("strike")
            origPriceList[i] = origPriceList[i].text
    
    return origPriceList

In [20]:
def getReleaseDate():
    """
    Get the product release date.
    
    Gets the release date of the products and returns a list of the release dates
    """
    releaseList = []
    
    # Märkte här hur mycket lättare det blev med att hämta datan från denna klass
    releaseList = soup.findAll("div", {"class", "responsive_search_name_combined"})
    
    for i in range(len(releaseList)):
        if(releaseList[i].find("div", {"class", "col search_released responsive_secondrow"})):
            releaseList[i] = releaseList[i].find("div", {"class", "col search_released responsive_secondrow"}).text
        else:
            releaseList[i] = "Unavailable"
    return releaseList
    
    #col search_released responsive_secondrow

## Data Scraping (and maybe Munging)

Vi filtrerar ut den info vi vill komma åt och sätter den info i respektive listor

In [21]:
gtList = []
for span in soup.findAll("span", {"class", "title"}):
    gtList.append(span.text)

reviewList = []
reviewList = soup.findAll("div", {"class", "responsive_search_name_combined"})
newList = reviewToNum(reviewList)

totalReviews = getTotalReviews()

discountList = []
listOfDiscounts = soup.findAll("div", {"class", "col search_discount responsive_secondrow"})
# loopa genom listan och rensa strängarna
for i, elem in enumerate(listOfDiscounts):
    discountList.append(elem.text)
    discountList[i] = re.sub("[^0-9]", '', elem.text)

origPriceList = getOrigPrice()

priceList = getProductPrice()

releaseList = getReleaseDate()

page = req.text
soup = bs(page, "html.parser")

### Vi sätter in datan i vår DataFrame

In [22]:
offersDf["Spelnamn"] = gtList
offersDf["Rating"] = newList
offersDf["#Reviews"] = totalReviews
offersDf["Rabatt%"] = discountList
offersDf["Pris"] = priceList
offersDf["OrdinariePris"] = origPriceList
offersDf["Utgivningsår"] = releaseList

# sätter in stöd av OS här på detta lata sätt
for i in range(len(allTitles)):
    if(allTitles[i].find("span", {"class", "platform_img win"})):
        offersDf["Win"][i] = "1"
    else:
        offersDf["Win"][i] = "0"
    
    if(allTitles[i].find("span", {"class", "platform_img mac"})):
        offersDf["OSX"][i] = "1"
    else:
        offersDf["OSX"][i] = "0"
    
    if(allTitles[i].find("span", {"class", "platform_img linux"})):
        offersDf["Lin"][i] = "1"
    else:
        offersDf["Lin"][i] = "0"

offersDf["Tid"] = tid

offersDf

,Spelnamn,Rating,#Reviews,Rabatt%,Pris,OrdinariePris,Utgivningsår,Win,Lin,OSX,Tid
0,Counter-Strike: Global Offensive,9,4962010,,Unavailable,Unavailable,"21 Aug, 2012",1,1,1,25.11.2020
1,DOOM Eternal,9,72663,67,"19,79€","59,99€","19 Mar, 2020",1,0,0,25.11.2020
2,The Sims™ 4,9,4727,88,"4,79€","39,99€","18 Jun, 2020",1,0,0,25.11.2020
3,theHunter: Call of the Wild™,9,48197,70,"5,99€","19,99€","16 Feb, 2017",1,0,0,25.11.2020
4,Red Dead Redemption 2,9,109781,33,"40,19€","59,99€","5 Dec, 2019",1,0,0,25.11.2020
...,...,...,...,...,...,...,...,...,...,...,...
120,The Forest,9,181693,50,"8,39€","16,79€","30 Apr, 2018",1,0,0,25.11.2020
121,Project CARS 2,7,8395,84,"9,59€","59,99€","21 Sep, 2017",1,0,0,25.11.2020
122,Grim Dawn,9,47168,75,"6,24€","24,99€","25 Feb, 2016",1,0,0,25.11.2020
123,Sid Meier's Civilization® VI: Gathering Storm,7,1718,60,"15,99€","39,99€","13 Feb, 2019",1,1,1,25.11.2020


### Vi skapar vår CSV-fil

In [23]:
# Här får du åtminstone lite pampesiska, njut :D 
# Och ööööh, vem behöver ens ta och hantera nå exceptions å använda 
# try och catch block när aaaalltid kommer att fungera så här felfritt? :O ;)

if os.path.exists("steamsale.csv"):
    print("he fanns rej så vi skapar int na nyyjje fil, men vi lägge till he du hämta i filn åtminstoni")
    offersDf.to_csv('steamsale.csv', sep=",", mode='a', header=False, index=False)
else:
    print("vi skapar en nyyjjan fil åt de me he data du har läst in, vassågoo")
    offersDf.to_csv('steamsale.csv', sep=",", index=False)

vi skapar en nyyjjan fil åt de me he data du har läst in, vassågoo


# Reflektion och feedback

Som förra projektet, jag njöt i stort sätt av denna också under alla delmoment.<br>Väldigt rolig faktiskt att jobba med och också skoj när man med tiden fick en allt bättre inblick i BeautifulSoup m.m.

Visst kunde mitt individuella projekt göras snyggare och mer optimerat, men vem har tid med det efter att man gjort så mycket och sedan efter insett vad som kan göras bättre ;) Huvudsaken är ju att projektet följer beskrivninge :D

Alla delmoment var riktigt skoja som sagt, ett bra projekt du tilldelat oss, hur "bra" du tycker att jag fullbordat det är en annan femma!